In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from zipfile import ZipFile
import wget
import keras
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator
import warnings
warnings.filterwarnings('ignore')

from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input, decode_predictions

In [2]:
#link = 'https://github.com/alexeygrigorev/dino-or-dragon/releases/download/data/dino-dragon.zip'
#data = wget.download(link)

#with ZipFile('./dino-dragon.zip', 'r') as zobject:
#    zobject.extractall('./dino-dragon')

In [3]:
path = './dino-dragon/train/dragon/0065e007-931f-4a67-ae27-7708e090cb26.jpg'
img = load_img(path)

In [4]:
def make_model():
        
    input = keras.Input(shape =(150,150,3))

    conv_1 = keras.layers.Conv2D(32, kernel_size = (3,3), activation = 'relu')(input)
    pooling_1 = keras.layers.pooling.MaxPool2D(pool_size = (2,2))(conv_1)
    flatten = keras.layers.Flatten()(pooling_1)
    dense_1 = keras.layers.Dense(64, activation = 'relu')(flatten)
    output = keras.layers.Dense(1, activation = 'sigmoid')(dense_1)

    model = keras.Model(input, output)
    losses = keras.losses.BinaryCrossentropy()
    optimizer = keras.optimizers.SGD(lr=0.002, momentum=0.8)

    model.compile(optimizer = optimizer, loss = losses, metrics = ['accuracy'])

    return model

In [5]:
train_gen = ImageDataGenerator(rescale= 1./255)
train_data = train_gen.flow_from_directory('./dino-dragon/train/',class_mode='binary', target_size = (150,150), batch_size = 20, shuffle = True)

test_gen = ImageDataGenerator(rescale= 1./255)
test_data = test_gen.flow_from_directory('./dino-dragon/test/', class_mode = 'binary', target_size = (150,150), batch_size = 20,shuffle = True)

Found 1594 images belonging to 2 classes.
Found 394 images belonging to 2 classes.


In [6]:
model = make_model()

history = model.fit(
    train_data,
    epochs=10,
    validation_data=test_data
    )

Epoch 1/10
80/80 [==============================] - 36s 440ms/step - loss: 0.6784 - accuracy: 0.5590 - val_loss: 0.6288 - val_accuracy: 0.7157
Epoch 2/10
80/80 [==============================] - 35s 439ms/step - loss: 0.5649 - accuracy: 0.7472 - val_loss: 0.4978 - val_accuracy: 0.7919
Epoch 3/10
80/80 [==============================] - 35s 434ms/step - loss: 0.4719 - accuracy: 0.7936 - val_loss: 0.4522 - val_accuracy: 0.7843
Epoch 4/10
80/80 [==============================] - 36s 444ms/step - loss: 0.4032 - accuracy: 0.8369 - val_loss: 0.4281 - val_accuracy: 0.8020
Epoch 5/10
80/80 [==============================] - 35s 438ms/step - loss: 0.3564 - accuracy: 0.8545 - val_loss: 0.3603 - val_accuracy: 0.8477
Epoch 6/10
80/80 [==============================] - 35s 440ms/step - loss: 0.3219 - accuracy: 0.8720 - val_loss: 0.3305 - val_accuracy: 0.8604
Epoch 7/10
80/80 [==============================] - 35s 431ms/step - loss: 0.3137 - accuracy: 0.8670 - val_loss: 0.3669 - val_accuracy: 0.8198

In [7]:
median_val = round(np.median(history.history['accuracy']),2)
print(median_val)

0.86


In [8]:
std = round(np.std(history.history['loss']),2)
print(std)

0.14


In [9]:
train_gen = ImageDataGenerator(rescale= 1./255,
                            rotation_range=40,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest'
                            )
train_data = train_gen.flow_from_directory('./dino-dragon/train/', class_mode = 'binary', target_size = (150,150), batch_size = 20, shuffle = True)

Found 1594 images belonging to 2 classes.


In [10]:
model = make_model()
history = model.fit(
    train_data,
    epochs=10,
    validation_data=test_data
    )

Epoch 1/10
80/80 [==============================] - 46s 570ms/step - loss: 0.6917 - accuracy: 0.5634 - val_loss: 0.6707 - val_accuracy: 0.5178
Epoch 2/10
80/80 [==============================] - 43s 533ms/step - loss: 0.6646 - accuracy: 0.5859 - val_loss: 0.6401 - val_accuracy: 0.6218
Epoch 3/10
80/80 [==============================] - 43s 541ms/step - loss: 0.6309 - accuracy: 0.6587 - val_loss: 0.5967 - val_accuracy: 0.6523
Epoch 4/10
80/80 [==============================] - 44s 554ms/step - loss: 0.6058 - accuracy: 0.6819 - val_loss: 0.5103 - val_accuracy: 0.7995
Epoch 5/10
80/80 [==============================] - 44s 546ms/step - loss: 0.5669 - accuracy: 0.7152 - val_loss: 0.4767 - val_accuracy: 0.7868
Epoch 6/10
80/80 [==============================] - 43s 530ms/step - loss: 0.5203 - accuracy: 0.7516 - val_loss: 0.4169 - val_accuracy: 0.8350
Epoch 7/10
80/80 [==============================] - 44s 548ms/step - loss: 0.5150 - accuracy: 0.7535 - val_loss: 0.4333 - val_accuracy: 0.7995

In [11]:
test_loss_mean = np.mean(history.history['val_loss'])
print(test_loss_mean)

0.5194456070661545


In [16]:
accuracy = history.history['val_accuracy']
np.mean(accuracy[5:])

0.7873096346855164